<a href="https://colab.research.google.com/github/pkpardeepkumar30/inline-java/blob/master/FV_Centred_Schemes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#! /usr/bin/env python
# -*- coding:utf-8 -*-

################################################################
#
# 1D-Advection problem
# Objective: Solve equation using the Centred Schemes
# Author: Pardeep Kumar
# Date: 01/08/2020
#
################################################################

#===============================================================
# Some libraries
#===============================================================

import numpy as np
import matplotlib.pyplot as plt

from matplotlib import animation, rc
from IPython.display import HTML


#===============================================================
# Some definitions
#===============================================================

scheme = 1                        # Chose your scheme: 1 (upwind), 2 (centered/Lax-Wendroff)
Nx = 101;                         # Number of grid points
xmax = 2.;                        # Domain limit to the right
xmin = -2.;                       # Domain limit to the left
dx = (xmax-xmin)/(Nx-1)           # Mesh size
dt = 0.04                         # Time step
c = 0.8                           # Advection speed
CFL = c*dt/dx                     # CFL number
x = np.linspace(xmin,xmax,Nx)       # Discretized mesh
#U = np.zeros(Nx-1)
  
U = np.exp( -0.5 * (x/0.4)**2 )   # Initial solution
U_old = U
Uex = U                           # Exact solution
t_end = 5.                        # Final time
Nt = int(t_end/dt)                # Number of iterations
t = np.linspace(0.,t_end,Nt+1)    # Time vector
alpha = 1.2;

def InitPlot(xlim, ylim):
    fig, ax = plt.subplots()
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

    line, = ax.plot([], [], lw=2)
    line2, = ax.plot([], [], lw=2, linestyle='dashed', marker='o')
    return fig, ax, line, line2

fig, ax, line1, line2 = InitPlot(( -2, 2), (0, 1.3))
#plt.scatter(x,U[0,:], marker='o', facecolors='white', color='k')

def init():
    line1.set_data([], [])
    line2.set_data([], [])
    return (line1,line2)

def Upwind(n):
    U_old = U.copy()
    if (c>0.):
      for i in range (1,Nx):
          U[i] = U_old[i] - CFL*(U_old[i]-U_old[i-1]);
      U[0] = U[Nx-1];
    else:
      for i in range (0,Nx-1):
          U[i] = U_old[i] - CFL*(U_old[i+1]-U_old[i]);
      U[Nx-1] = U[0];
    
    line1.set_data(x, U)
    return line1


def Lax_Friedrichs(n):
    U_old = U.copy()

    for i in range(0, Nx-1):
        U[i] = 0.5*(U_old[i+1] + U_old[i-1]) - (0.5*c*dt/dx)*(U_old[i+1]-U_old[i-1]);
    
    U[Nx-1] = 0.5*(U_old[0] + U_old[Nx-2]) - (0.5*c*dt/dx)*(U_old[0]-U_old[Nx-2]);
    line1.set_data(x, U)
    return line1

def Lax_Friedrichs_Flux(j, F, U_):
    return 0.5 * (F[j] + F[j+1]) - 0.5 * (dx/dt)*(U_[j+1] - U_[j])

def ForceFlux(j, F, U_):
    UnHalf = 0.5*(U_[j] + U_[j+1]) - 0.5*(dt/dx)*(F[j+1] - F[j])
    FnHalf = c * UnHalf

    return 0.25 * (F[j] + 2.0*FnHalf + F[j+1] - (dt/dx)*(U_[j+1] - U_[j]))

def ForceFlux_2(F2, F1, U2, U1):
    UnHalf = 0.5*(U1 + U2) - 0.5*(dt/dx)*(F2 - F1)
    FnHalf = c * UnHalf

    return 0.25 * (F1 + 2.0*FnHalf + F2 - (dt/dx)*(U2 - U1))

def Musta_Flux(i, F, U_):
    Q2i = U_[i] - alpha*(dt/dx) * (ForceFlux(i, F, U_) - c* U_[i])
    Q2iPlus1 = U_[i+1] - alpha*(dt/dx) * (c* U_[i+1] - ForceFlux(i, F, U_))

    #return ForceFlux_2(c*Q2iPlus1, c*Q2i, Q2iPlus1, Q2i)
    UnHalf = 0.5*(Q2i + Q2iPlus1) - 0.5*(dt/dx)*(c*Q2iPlus1 - c*Q2i)
    FnHalf = c * UnHalf

    return 0.25 * (c*Q2i + 2*FnHalf + c*Q2iPlus1 - (dt/dx)*(Q2iPlus1 - Q2i))


def Scheme(FluxFunc, n):
    F = c * U;
    U_old = U.copy()

    for i in range(0, Nx-1):
        U[i] = U_old[i] - (dt/dx) * (FluxFunc(i, F, U_old) - FluxFunc(i-1, F, U_old))
    
    U[Nx-1] = 0.5*(U_old[0] + U_old[Nx-2]) - (0.5*c*dt/dx)*(U_old[0]-U_old[Nx-2]);
    line1.set_data(x, U)

    return line1

def Scheme2(FluxFunc, n):
    F = c * U;
    U_old = U.copy()

    for i in range(0, Nx-1):
        U[i] = U_old[i] - (dt/dx) * (FluxFunc(F[i+1], F[i], U_old[i+1], U_old[i]) - FluxFunc(F[i], F[i-1], U_old[i], U_old[i-1]))
    
    U[Nx-1] = 0.5*(U_old[0] + U_old[Nx-2]) - (0.5*c*dt/dx)*(U_old[0]-U_old[Nx-2]);
    line1.set_data(x, U)

    return line1 

def Analytical(n):
    d = c*n*dt
    Uex = np.exp(-0.5*(np.mod(x-d+xmax,4)-xmax)**2/0.4**2)
    #errL1 = U - Uex
    #errL2 = np.linalg.norm(errL1)
    
    line2.set_data(x, Uex)
    return line2

def Step2(n):
    return Scheme(Musta_Flux, n), Analytical(n)

def Step(n):
    global U
    if (scheme == 1):
    
      Un = U
      if (c>0.):
          Um = np.roll(Un,1)
          U = Un - CFL*(Un-Um)
      else:
          Up = np.roll(Un,-1)
          U = Un - CFL*(Up-Un)
          
    # Solve equation using the centered scheme with/without dissipation
    if (scheme == 2):
  
      theta = (c*dt/dx)**2;
      Un = U
      Um = np.roll(Un,1)
      Up = np.roll(Un,-1)
      U  = Un - 0.5 * CFL* (-Up-Um) + 0.5*theta*(Up-2*Un+Um)
      
  #===============================================================
  # Compute exact solution
  #===============================================================
    d = c*n*dt
    Uex = np.exp(-0.5*(np.mod(x-d+xmax,4)-xmax)**2/0.4**2)
    errL1 = U - Uex
    errL2 = np.linalg.norm(errL1)
    line1.set_data(x, U)
    line2.set_data(x, Uex)
    return (line1, line2)

# call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, Step2, init_func=init,
                               frames=len(t), interval=40, blit=True)

# equivalent to rcParams['animation.html'] = 'html5'
rc('animation', html='html5')
anim